![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [28]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [29]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [30]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [31]:
# Limpieza de data

import re
from string import digits
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

def clean_text(texto):
    
    texto = re.sub("\'", "", texto) 
    texto = re.sub("[^a-zA-Z]"," ",texto)
    texto = texto.lower() 
    
    remove_digits = str.maketrans('', '', digits)
    texto = texto.translate(remove_digits)
    
    return texto

dataTraining['plot'] = dataTraining['plot'].apply(lambda x: clean_text(x))


def tokenization(texto):
    texto = re.split('\W+', texto)
    return texto

dataTraining['plot'] = dataTraining['plot'].apply(lambda x: tokenization(x.lower()))

def stemming(texto):
    text = [stemmer.stem(word) for word in texto]
    return texto

dataTraining['plot'] = dataTraining['plot'].apply(lambda x: stemming(x))

dataTraining['plot']=[" ".join(plot) for plot in dataTraining['plot'].values]
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,in sweden a female blackmailer with a disfigur...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,in a friday afternoon in new york the presiden...,['Drama'],7.4
2582,1990,Narrow Margin,in los angeles the editor of a publishing hous...,"['Action', 'Crime', 'Thriller']",6.6


### CountVectorizer

In [6]:
# Definición de variables predictoras (X)
vect = CountVectorizer(max_features=5000, stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 5000)

In [7]:
dataTraining.head(10)

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,in sweden a female blackmailer with a disfigur...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,in a friday afternoon in new york the presiden...,['Drama'],7.4
2582,1990,Narrow Margin,in los angeles the editor of a publishing hous...,"['Action', 'Crime', 'Thriller']",6.6
9784,1943,The Ox-Bow Incident,two drifters are passing through a western tow...,"['Drama', 'Western']",8.1
8804,2004,Hellbent,two gay men on a date are murdered the night b...,"['Horror', 'Thriller']",5.5
3501,1941,How Green Was My Valley,adult huw morgan is about to move away from th...,"['Drama', 'Family']",7.8
7145,2008,Ghost Town,bertram pincus is a man whose people skills le...,"['Comedy', 'Drama', 'Fantasy', 'Romance']",6.7
4067,1996,Kingpin,roy munson was raised to be the best bowler in...,"['Comedy', 'Sport']",6.9


### TfidfVectorizer

In [32]:
vect = TfidfVectorizer(max_features=10000)
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 10000)

In [33]:
dataTraining.head(10)

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,in sweden a female blackmailer with a disfigur...,"['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,in a friday afternoon in new york the presiden...,['Drama'],7.4
2582,1990,Narrow Margin,in los angeles the editor of a publishing hous...,"['Action', 'Crime', 'Thriller']",6.6
9784,1943,The Ox-Bow Incident,two drifters are passing through a western tow...,"['Drama', 'Western']",8.1
8804,2004,Hellbent,two gay men on a date are murdered the night b...,"['Horror', 'Thriller']",5.5
3501,1941,How Green Was My Valley,adult huw morgan is about to move away from th...,"['Drama', 'Family']",7.8
7145,2008,Ghost Town,bertram pincus is a man whose people skills le...,"['Comedy', 'Drama', 'Fantasy', 'Romance']",6.7
4067,1996,Kingpin,roy munson was raised to be the best bowler in...,"['Comedy', 'Sport']",6.9


In [34]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [35]:
y_genres.shape

(7895, 24)

In [36]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

## Random Forest

In [37]:
# Definición y entrenamiento
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=10,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                              

In [38]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.811770019789752

## Logistic Regression with Multilabel

In [39]:
# Definición y entrenamiento
clf = OneVsRestClassifier(LogisticRegression(penalty='l2', solver='lbfgs', random_state=1))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=1, solver='lbfgs',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

In [40]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.8764323961240793

## Naive Bayes

In [49]:
# Definición y entrenamiento
clf = OneVsRestClassifier(MultinomialNB())
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                            fit_prior=True),
                    n_jobs=None)

In [50]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.528446407763295

## XGBoost

In [41]:
# Definición y entrenamiento
clf = OneVsRestClassifier(XGBClassifier(learning_rate=0.1, colsample_bytree=0.5, gamma=4, random_state=1))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=4,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
          

In [42]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.8309429211183721

## LSTM 

In [43]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from livelossplot import PlotLossesKeras
%matplotlib inline

In [44]:
X_LTSM = dataTraining['plot'].tolist()

# For vocabulary only the intersec characters is used to avoid issues with data collection
voc = set(''.join(X_LTSM))
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}

# Convert characters to int and pad
X_LTSM = [[vocabulary[x1] for x1 in x if x1 in vocabulary.keys()] for x in X_LTSM]
X_LTSM = sequence.pad_sequences(X_LTSM)

# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_LTSM, y_genres, test_size=0.33, random_state=42)

In [52]:
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128)) #input_length=max_len
model.add(LSTM(21, dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(24, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary() 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         3584      
                                                                 
 lstm_2 (LSTM)               (None, 21)                12600     
                                                                 
 dropout_2 (Dropout)         (None, 21)                0         
                                                                 
 dense_2 (Dense)             (None, 24)                528       
                                                                 
Total params: 16,712
Trainable params: 16,712
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train_genres, validation_data=[X_test, y_test_genres], batch_size=24, epochs=10, verbose=1, callbacks=[PlotLossesKeras()])

Epoch 1/10
  2/221 [..............................] - ETA: 1:15:33 - loss: 0.6917 - accuracy: 0.0417

## Test

In [36]:
# transformación variables predictoras X del conjunto de test

dataTesting['plot'] = dataTesting['plot'].apply(lambda x: clean_text(x))
dataTesting['plot'] = dataTesting['plot'].apply(lambda x: tokenization(x))
dataTesting['plot'] = dataTesting['plot'].apply(lambda x: stemming(x))

X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [37]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.130461,0.104165,0.023082,0.036926,0.362043,0.140937,0.040516,0.500756,0.075848,0.083541,...,0.043975,0.074926,0.000203,0.296664,0.064354,0.007702,0.020413,0.211857,0.028172,0.019066
4,0.131418,0.125748,0.025362,0.041335,0.369488,0.170411,0.044278,0.519344,0.076322,0.070106,...,0.025407,0.074518,0.000387,0.181621,0.064354,0.008648,0.021135,0.219412,0.030101,0.019214
5,0.157803,0.127933,0.023082,0.060387,0.303800,0.395183,0.046342,0.624845,0.080504,0.083046,...,0.060180,0.170291,0.000410,0.275781,0.084491,0.010109,0.022811,0.366414,0.032481,0.018760
6,0.155076,0.112635,0.023082,0.068116,0.331942,0.146852,0.054136,0.541919,0.067277,0.079445,...,0.044700,0.107194,0.000054,0.247511,0.091870,0.007166,0.043342,0.254286,0.040614,0.019214
7,0.174799,0.135999,0.025174,0.038526,0.336406,0.192343,0.040939,0.405001,0.073752,0.118732,...,0.025407,0.090971,0.000143,0.202324,0.236505,0.008569,0.030016,0.244904,0.037864,0.019080
